In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.5 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00


In [ ]:
!pip install rouge

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from sklearn.model_selection import train_test_split
data = pd.read_excel(r'/content/drive/MyDrive/bart/MeQSum_ACL2019_BenAbacha_Demner-Fushman.xlsx')

train_data, temp = train_test_split(data, test_size=0.2, random_state=42)
test_data, val_data = train_test_split(temp, test_size=0.5, random_state=42)

train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [ ]:
class T5CustomDataset(Dataset):

    def __init__(self, dataframe, source_col, target_col, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_col = source_col
        self.target_col = target_col
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source = "summarize: " + str(self.data[self.source_col][index])
        target = str(self.data[self.target_col][index])

        source = self.tokenizer.encode_plus(
            source,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt",
             truncation=True
        )

        target = self.tokenizer.encode_plus(
            target,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt",
             truncation=True
        )

        return {
            "input_ids": source["input_ids"].flatten(),
            "attention_mask": source["attention_mask"].flatten(),
            "labels": target["input_ids"].flatten(),
        }

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
train_dataset_t5 = T5CustomDataset(train_data, 'CHQ', 'Summary', tokenizer, 1000)
val_dataset_t5 = T5CustomDataset(val_data, 'CHQ', 'Summary', tokenizer, 1000)
test_dataset_t5 = T5CustomDataset(test_data, 'CHQ', 'Summary', tokenizer, 1000)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [ ]:
import numpy as np
best_loss = np.inf
best_batchsize = None
best_epoch = None

for batchsize in range(4,5):
    torch.cuda.empty_cache()
    train_dataloader = DataLoader(train_dataset_t5, batch_size=batchsize)
    val_dataloader = DataLoader(val_dataset_t5, batch_size=batchsize)

    optimizer = AdamW(model.parameters(), lr=1e-5)
    total_steps = len(train_dataloader) * 2
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(1):
        print(f'Epoch: {epoch+1}')

        # Training Phase
        model.train()
        total_train_loss = 0
        for batch in train_dataloader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            total_train_loss += loss.item()

        # Validation Phase
        model.eval()
        total_val_loss = 0
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f'Validation Loss: {avg_val_loss}')

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            best_batchsize = batchsize
            best_epoch = epoch + 1
            model_path = "/content/drive/MyDrive/t5/best_model"
            model.save_pretrained(model_path)
            print(f'Saving model at epoch {best_epoch} with validation loss of {best_loss:.3f}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 1
Validation Loss: 0.808543880879879
Saving model at epoch 1 with validation loss of 0.809


In [ ]:
model_path = "/content/drive/MyDrive/t5/best_model"
model = T5ForConditionalGeneration.from_pretrained(model_path)
test_dataset_t5 = T5CustomDataset(test_data, 'CHQ', 'Summary', tokenizer, 1000)
test_dataloader = DataLoader(test_dataset_t5, batch_size=4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
model.eval()
predictions = []

for batch in test_dataloader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in outputs]
    predictions.extend(preds)




In [ ]:
print(predictions)

['i have been transfusing every 22 days . i heard about a tablet', 'i have giant cell arteritis. on steroids. lost taste . i', 'shingles MESSAGE: I am having symptoms of shingles, no rash or blister', 'my grandson has contracted Rubella . he has had at least one of the vaccine', 'Ochoa syndrome MESSAGE: Where could I find more information about the rare disease', 'HYPOMATURATION TYPE, IIA2 I have 2 other daughters and', 'if you have any questions, please contact me .', 'garcinia cambodia is a plant that promotes the extract of this plant', 'a friend is 35 and his sugar level is around 100 or 120 . he tests', 'i need it fixed . it has been broken for 3 years .', 'my left hand has not been moving like it should MESSAGE . my arm itself seems', 'lipoma of forearm MESSAGE has returned .', 'starch is the source of starch . is it safe for someone with Celiac', 'clonidine is a drug that is not prescribed for a three year old female', 'chronic myeloid leukemia (CML) is a chronic myeloid le', 'a

In [ ]:

reference_summaries = test_data['Summary'].tolist()
from rouge import Rouge

rouge = Rouge()
rouge_l_scores = rouge.get_scores(predictions, reference_summaries, avg=True)['rouge-l']

print("ROUGE-L Scores:", rouge_l_scores)

from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('punkt')
nltk.download('wordnet')
# Tokenize the reference summaries and predictions
tokenized_reference_summaries = [word_tokenize(ref) for ref in reference_summaries]
tokenized_predictions = [word_tokenize(pred) for pred in predictions]

# Compute METEOR scores
meteor_scores = [meteor_score([ref], pred) for ref, pred in zip(tokenized_reference_summaries, tokenized_predictions)]
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)

print("Average METEOR Score:", avg_meteor_score)

from nltk.translate.bleu_score import corpus_bleu

# Format the reference summaries for use with nltk's corpus_bleu
references = [[ref.split()] for ref in reference_summaries]
# Tokenize the generated predictions
candidates = [pred.split() for pred in predictions]

bleu_score = corpus_bleu(references, candidates)

print("BLEU Score:", bleu_score)

ROUGE-L Scores: {'r': 0.11463278469896113, 'p': 0.10653359140859138, 'f': 0.10544981113771486}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Average METEOR Score: 0.1467251726952368
BLEU Score: 0.021479530820022925
